In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd
import seaborn as sns# data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# IESB
## Pós Graduação em Ciência de Dados

### **Disciplina** - Data Mining e Machine Learning II
### **Projeto de Conclusão da Disciplina **
### **Aluno:** Nilson Romero Michiles Júnior
### **Turma:** Asa Norte

 
#### Descrição do Problema e Objetivo

Os Bancos possuem uma necessidade de manter seu percentual de inadimplência baixo, sendo mais oneroso ainda tentar recuperar créditos de habitação(Home Equity), tendo em vista as proteções que a legislação concede aos inadimplentes dessa modalidade. Visto isso, urge a necessidade dos Bancos atuarem preventivamente para identificar possíveis maus pagadores e evitar possíveis adversidades futuras. 

Assim, esse modelo de dados apresenta preditor para identificar esses maus pagadores, por meio de um conjunto de dados de Home Equity com   cerca de seis mil empréstimos concedidos no passado. Esses dados contêm várias informações sobre a situação do cliente no momento do empréstimo e também contêm uma coluna 'RUIM', que indica se o cliente deixou de pagar o empréstimo posteriormente. Podemos usar esse conjunto de dados juntamente com a variável/etiqueta "RUIM" para treinar modelos de aprendizado de máquina, o que nos ajudaria a prever a probabilidade de alguém deixar o empréstimo no futuro com base na situação atual. 

Assim, ao final deste notebook será proposto o melhor modelo a ser usado para prever a etiqueta supramencionada baseada no padrão da situação apresentada nos dados. Esse problema pode ser classificado como problema de classificação binária, pois o modelo preverá se uma pessoa seria o padrão.


#### Metodologia

A base de dados "Home Equity" possui dados pessoas e informações de empréstimo de 5.960 empréstimos recentes. Para cada empréstimo existem 12 variáveis registradas. A variável alvo (BAD) indica quando o cliente não pagou o empréstimo (valor 1), e quando ele honrou o compromisso (valor 0).

Serão utilizados os modelos Random Forest Classifier, XGBosst e XGBoost com auxílio do GridSearchCV para otimização do modelo

### Dicionário de Dados
O dicionário de dados das colunas disponíveis no Dataset estão elencadas abaixo:

**BAD:** 1 = client defaulted on loan 0 = loan repaid

**LOAN**: Amount of the loan request

**MORTDUE**: Amount due on existing mortgage

**VALUE**: Value of current property

**REASON**: DebtCon = debt consolidation ; 
HomeImp = home improvement

**JOB**: Six occupational categories

**YOJ**: Years at present job

**DEROG**: Number of major derogatory reports

**DELINQ**: Number of delinquent credit lines

**CLAGE**: Age of oldest trade line in months

**NINQ**: Number of recent credit lines

**CLNO**: Number of credit lines

**DEBTINC**: Debt-to-income ratio

### Importando o Dataset 

In [ ]:
df = pd.read_csv('/kaggle/input/hmeq-data/hmeq.csv')
df.head()

### Exploração do Dataset

In [ ]:
df.shape, df.info()

Observa-se que Dataset possui 5960 linhase  13 colunas, sendo apenas as colunas 'JOB' e 'REASON' com valores não númericos

In [ ]:
# Estatísticas Descritivas
df.describe(include='all')

Para a análise inicial, há a biblioteca Pandas Profiling que gera um report com análise de todas os campos e suas estatísticas. Por meio deste relatório é possível ter uma noção das distribuiçoes dos dados e correlações

In [ ]:
#Utilizando o pandas profiling para auxiliar a EDA
import pandas_profiling as pp
pp.ProfileReport(df)

In [ ]:
# avaliação das variáveis numéricas por meio de histogramas
import matplotlib.pyplot as plt
%matplotlib inline
df.hist(figsize=(20,10))

Na análise inicial, se observa que a variável Target('BAD'), possui um número pequeno de 'maus pagadores'(=1), o que indica que é uma base desbalanceada, sendo necessários ajustes ou métricas específicas para essa distribuição

## Tratamento dos Dados

Para o tratamento, serão avaliadas a existência de missing values ou valores null.

In [ ]:
MissingValues =df.isnull().sum().rename_axis('Colunas').reset_index(name='Missing Values')
MissingValues

Pelo quadro acima, se observa que a base de dados não foi tratada, havendo uma quantidade alta de missings. Para a análise foram excluídos as linhas que possuiam algum valor com NA, restando 3364 linhas.

In [ ]:
# retirando os na
df2 = df.copy()
df2.dropna(axis=0,how='any',inplace= True)
df2.info(), df2.isna().any() 


## Análise Descritiva Exploratória (EDA)



In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
df2.hist(figsize=(25,14),bins=10)

A avaliação dos histogramas mostra inicialmente que :
- A variável BAD (Target) possui poucos valores 1 para treinamento do modelo
- A maior parte dos valores totais de financiamento (LOAN) possuem uma distribuição próxima da normalidade, e os valores a receber(MORTDUE), na média, são maiores que os totais emprestados. Observa-se o terror dos juros bancários
- Os valores das propriedades possuem distribuição próxima dos valores dos financiamentos
- O DEROG, algo equivalente à um aviso de negativação do serviço de proteção ao consumidor, é baixo, contudo possui uma correlação próxima de moderada(p=0,25) com os maus pagadores.
- O DELINQ, linhas de crédito com inadimplência, também possui correlação próxima à moderada(p=0,27) com maus pagadores
- O número de linhas de crédito possui correlação, mas a intensidade é menor(p=0,13), com maus pagadores
- Por fim, a base possui um indicador (Débitos/Renda) que possui uma correlação próxima a moderada (p=0,23), sendo um bom indicador.

In [ ]:
# Correlação das variáveis numéricas
plt.figure(figsize= (15, 15))

sns.heatmap(df2.corr(), square=True, annot=True, linewidth=0.5)

In [ ]:
dfWithBin = df.copy()
bins=[0,3,15] 
group=['Low','High'] 
dfWithBin['DELINQ_bin']=pd.cut(dfWithBin['DELINQ'],bins,labels=group)
LOAN_bin=pd.crosstab(dfWithBin['DELINQ_bin'],dfWithBin['BAD'])
LOAN_bin.div(LOAN_bin.sum(1).astype(float), axis=0).plot(kind="bar", stacked=True,title='Cruzamento de Linhas de Crédito Inadimplentes e Maus pagadores')
plt.xlabel('DELINQ')
P= plt.ylabel('%')

A título de demonstração da correlação entre os valores, o gráfico acima explicita que um número alto de linhas de crédito inadimplentes tem relação positiva com maus pagadores.

Se observa que, os devedores(BAD)  em média fizeram empréstimos de 19.260,00, contudo sua dívida em média está em $ 73.864,00.

In [ ]:
#avaliacao dos default loans

df2[df2['BAD']==1].drop('BAD', axis=1).describe().style.format("{:.2f}")

In [ ]:
# Avaliando as variáveis categóricas em relacao ao pefil do pagador

JOB=pd.crosstab(df['JOB'],df['BAD'])
JOB.div(JOB.sum(1).astype(float), axis=0).plot(kind="bar", stacked=True, title='Tipos de Empregos e Clientes', figsize=(4,4))

Pela análise, se observa que o grupo que trabalha com vendas e empreendedores possuem um número maior de maus pagadores

In [ ]:
REASON=pd.crosstab(df['REASON'],df['BAD'])
REASON.div(REASON.sum(1).astype(float), axis=0).plot(kind="bar", stacked=True, title='Tipos de Empregos e Razões', figsize=(4,4))

A variável razão do débito apresentou valores próximos para as duas categorias, o que não dá muita informação ao modelo

### Featuring Engineering

Para melhor ajuste ao modelo, foram dummizadas as tabelas com type Object

In [ ]:
# Gerando Dummies para modelos que utilizam apenas variaveis numéricas

df2 = pd.get_dummies(df2, columns=['REASON', 'JOB'])

In [ ]:
df2.head().T

In [ ]:
#Normalizando os dados para facilitar possível visualizacoes

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
df3 = pd.DataFrame(sc.fit_transform(df2), columns=df2.columns)

## Geração Amostras de Treino e Teste

Nota: Neste trabalho foi realizada a modelagem utilzando uma amostra para validação inclusive, contudo, devida a baixa quantidade de registros, foi utilizado apenas treino e teste 

In [ ]:
# importando a biblioteca
from sklearn.model_selection import train_test_split

In [ ]:
#Etapa 1- Primeiro Separando em Treino e Teste, parâmetro test_size = 0.25 (default)
treino, teste = train_test_split(df2, random_state=42)

#Etapa 2 -  Separando o Treino em treino e validacao, para refinar o modelo
#treino, validacao = train_test_split(treino, random_state=42)

treino.shape, teste.shape # validacao.shape, 

In [ ]:
teste.describe()

In [ ]:
# Verificando se as amostras possuem similaridade, avaliando se há discrepância alta considerando a média e desvio padrão de cada uma. Pela análise verifica-se que a amostra gerada 
# possuem estatísticas próximas, portanto atendem ao requisito.
treino.describe()

In [ ]:
#Selecionando as colunas que usaremos para treinar o modelo
nao_usadas = ['BAD']

# Lista das colunas que serão usadas
usadas = [c for c in treino.columns if c not in nao_usadas]

## Métricas de Avaliação

Para a avaliação do modelo serão utilizadas duas métricas,sendo a Accuracy(Acurácia) e o F1 Score, melhor detalhados abaixo:

<img src="https://miro.medium.com/max/1000/1*t1vf-ofJrJqtmam0KSn3EQ.png" width="250px"/>

A **Accuracy** mede a performance do modelo como um todo, contudo não é uma métrica interessante em situações de bases muito desbalanceadas.

A **Precision** é importante quando os Falsos Positivos são considerados mais prejudiciais que os Falsos Negativos. Sendo uma métrica interessante para o modelo em análise caso o apetite à risco do Banco seja baixo, logo se para o Banco acertar na predição dos maus pagadores seja mais importante que acabar deixando de emprestar para algum bom pagador que o modelo etiquetou errado.

O **Recall**, ao contrário,  pode ser usada em situações em que os Falsos Negativos são mais prejudiciais que os Falso Positivos. Nesse sentido, o foco seria ter mais produtos financiamentos aprovados, assim, o Banco sofre mais deixando de vender para os bons pagadores do que aceitando um mau pagador etiquetado como bom.

O **F1 Score** é uma média harmônica entre de Precision e Recall, portanto, quando tem-se um F1-Score baixo, é um indicativo de que ou a precisão ou o recall está baixo.






In [ ]:
# Avaliando desempenho do modelo
#importando métrica
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
results = pd.DataFrame(columns=['Modelo', 'Accuracy', 'F1score'])

## Modelo RandomForest Classifier

As Florestas aleatórias ou florestas de decisão aleatória são modelos ensemble das DecisionTree, que utilizam um método de aprendizado conjunto para classificação,regressão e outras tarefas que operam construindo uma infinidade de árvores de decisão no momento do treinamento e gerando a classe que é o modo das classes ou a previsão média das árvores individuais.

In [ ]:
# importanto o modelo
from sklearn.ensemble import RandomForestClassifier

#instanciando o modelo
rf = RandomForestClassifier(n_estimators=200,random_state=42)

In [ ]:
# treinando o modelo
rf.fit(treino[usadas], treino['BAD'])

#Prevendo os dados de validacao

# gerando predicoes do modelo com os dados de teste
pred_teste = rf.predict(teste[usadas])

#Medindo a acuracia nos dados de teste
results.loc[0]= ['RandonForest sem ajuste', accuracy_score(teste['BAD'],pred_teste), f1_score(teste['BAD'],pred_teste)]

accuracy_score(teste['BAD'],pred_teste), f1_score(teste['BAD'],pred_teste)


In [ ]:
# Avaliando a importancia de cada coluna (cada variável de entrada)
pd.Series(rf.feature_importances_, index=usadas).sort_values().plot.barh()

In [ ]:
# importando a bilbioteca para plotar o gráfico de Matriz de Confusão
import scikitplot as skplt

# Matriz de Confusão - Dados de Validação
skplt.metrics.plot_confusion_matrix(teste['BAD'], pred_teste)


Pela análise da Matriz de Confusão, considerando que é uma base bastante desbalanceada, como se observa no gráfico abaixo. Assim, a análise das métricas de especifidade e esforço pode realçar os falsos positivos

### Utilizando o RandonForest Classifier com ajuste nos parâmetros

Foram ajustados os parâmetros de aumentando o número de estimadores para 900, quando o default é 100, e informando que o número de folhas aceitavel para as ramificações das árvores de decisão como 2.


In [ ]:
# Setando parametros
rf2 = RandomForestClassifier(max_depth=None, random_state=42, n_jobs=-1, n_estimators=900,
                            min_impurity_decrease=1e-3, min_samples_leaf=2,  class_weight='balanced')
# treinando o modelo RF2
rf2.fit(treino[usadas], treino['BAD'])

In [ ]:
#relizando a predicao do RF2 com base teste
pred_teste2 = rf2.predict(teste[usadas])

#métrica para RF2 validacao
results.loc[1]= ['RandonForest COM ajuste', accuracy_score(teste['BAD'],pred_teste2), f1_score(teste['BAD'],pred_teste2)]

accuracy_score(teste['BAD'],pred_teste2), f1_score(teste['BAD'],pred_teste2)

In [ ]:
# Matriz de Confusão - Dados de Validação
skplt.metrics.plot_confusion_matrix(teste['BAD'], pred_teste2)

Com os ajustes de parâmetros, houve uma melhora pequena no F1 Score.

## Modelo XGBoost

O XGBoost é uma implementação de árvores de decisão aprimoradas por gradiente, projetadas para velocidade e desempenho.Sua sigla significa eXtreme Gradient Boosting, e sua vantagem é devida a uma implementação de máquinas de aumento de gradiente.

In [ ]:
# Importar o modelo
from xgboost import XGBClassifier

# Instanciar o modelo
xgb = XGBClassifier(n_jobs=-1, random_state=42)

# treinando o modelo
xgb.fit(treino[usadas],treino['BAD']) 

# Fazendo predições
#pred_xgb_validacao = xgb.predict(validacao[usadas])

# Metrícas XGB validacao
#accuracy_score(validacao['BAD'],pred_xgb_validacao), balanced_accuracy_score(validacao['BAD'],pred_xgb_validacao), f1_score(validacao['BAD'],pred_xgb_validacao)

In [ ]:
# Fazendo predições
pred_xgb_teste = xgb.predict(teste[usadas])

# Metrícas XGB teste
results.loc[2]= ['XGBoost', accuracy_score(teste['BAD'],pred_xgb_teste), f1_score(teste['BAD'],pred_xgb_teste)]

accuracy_score(teste['BAD'],pred_xgb_teste), f1_score(teste['BAD'],pred_xgb_teste)

In [ ]:
# Matriz de Confusão - Dados de Validação
skplt.metrics.plot_confusion_matrix(teste['BAD'], pred_xgb_teste)

O XGBoost apresentou uma melhora um pouco maior no modelo, contudo não foi significante.

## Modelo XGBoost com GridSearchCV

O GridSearchCV é um módulo do Scikit Learn que é amplamente usado para automatizar grande parte do processo de tuning. O objetivo primário do GridSearchCV é a criação de combinações de parâmetros para posteriormente avaliá-las.

In [ ]:
# Importação bibliotecas
# Importação GridSearchCV.
from sklearn.model_selection import GridSearchCV

# Uso do constructor do XGBoost para criar um classifier.
xgb2 = XGBClassifier(n_jobs=-1) # Sem nada dentro, pois vamos "variar" os parâmetros.

In [ ]:
# Para o balaceamento do gridSearchCV foram realizadas três rodadas, a partir dos best score de cada época. 
parametros = {'n_estimators':[100,500, 900, 1100],
              'learning_rate':[0.02,0.08,0.09,1.5]}

In [ ]:
# Importando o Make Scorer
from sklearn.metrics import make_scorer

# Importando os módulos de cálculo de métricas
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [ ]:
# Criando um dicionário com as métricas que desejo calcular.
meus_scores = {'accuracy' :make_scorer(accuracy_score),
               'recall'   :make_scorer(recall_score),
               'precision':make_scorer(precision_score),
               'f1'       :make_scorer(f1_score)}

# Exemplo para o uso scoring igual ao meus_scores.
grid = GridSearchCV(estimator = xgb2,
                      param_grid = parametros,
                      cv = 10,
                      scoring = meus_scores,   # É o meus_scores
                      refit = 'f1')            # Observe que foi configurado para f1

# Imprime o melhor score(f1) e melhor parâmetro 
grid.fit(treino[usadas],treino['BAD'])

In [ ]:
grid.best_score_, grid.best_params_

In [ ]:
#  Caso queira dar uma olhada nos outros scores
pd.DataFrame(grid.cv_results_).sort_values('rank_test_f1')[:3].T

In [ ]:
# Criando um objeto que os melhores parametros.
xgb_gs = grid.best_estimator_

# Visualizar o objeto para conferir os parametros.
xgb_gs

In [ ]:
#primeira epoca 
# Fazendo predições teste
pred_xgb_gs_teste = xgb_gs.predict(teste[usadas])

# Metrícas XGB teste
results.loc[3]= ['XGBoost com GridSearchCV',accuracy_score(teste['BAD'],pred_xgb_gs_teste), f1_score(teste['BAD'],pred_xgb_gs_teste)]

accuracy_score(teste['BAD'],pred_xgb_gs_teste), f1_score(teste['BAD'],pred_xgb_gs_teste)

In [ ]:
# Matriz de Confusão - Dados de Validação
skplt.metrics.plot_confusion_matrix(teste['BAD'], pred_xgb_gs_teste)

## Modelo Random Forest Classifier e GridSearchCV 

Por fim, a título de avaliação, foi testada a Random Forest com tuning pelo GridSearchCV para verifiar possível melhora no modelo.

In [ ]:
#instanciando o modelo
rf2=RandomForestClassifier(n_jobs=-1)

#setando parametros para o gridSearchCV
param_dict = { 'n_estimators':[100,400,800,1000],
               'criterion': ['gini','entropy']
              }

grid2 = GridSearchCV(rf2, param_dict, cv=10)

#treinando modelo
grid2.fit(treino[usadas], treino['BAD'])

In [ ]:
#Resultados
grid2.best_params_ , grid2.best_score_

In [ ]:
# Criando um objeto que os melhores parametros.
rf2_gs2 = grid2.best_estimator_

# Visualizar o objeto para conferir os parametros.
rf2_gs2


In [ ]:
# predicao teste
pred_rf2_gs2_teste = rf2_gs2.predict(teste[usadas])

# metricas predicao teste
results.loc[4]= ['RandomForest com GridSearchCV', accuracy_score(teste['BAD'],pred_rf2_gs2_teste),f1_score(teste['BAD'],pred_rf2_gs2_teste)]

accuracy_score(teste['BAD'],pred_rf2_gs2_teste), f1_score(teste['BAD'],pred_rf2_gs2_teste)

In [ ]:
# Matriz de Confusão - Dados de Validação
skplt.metrics.plot_confusion_matrix(teste['BAD'], pred_rf2_gs2_teste)

# Conclusão

**Pela Análise dos Modelos, verifica-se que o que apresentou o melhor resultado foi XGBoost, com as métricas de acurária 0.958 e F1 Score 0.6846)**

In [ ]:
results